Used to generate XGB predictions

In [10]:
import xgboost
from xgboost import XGBRegressor, XGBClassifier

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error, r2_score, mean_squared_error

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from category_encoders import MEstimateEncoder, TargetEncoder
from sklearn.inspection import permutation_importance

import seaborn as sns
import shap



from xgb_utils import get_na_cols, xgb_params, categorical_cols, one_hot_cols, target_encoding_cols, ordinal_cols, useless_cols, numeric_cols

In [11]:
process_train_path = '../output/processed_train.csv'
test_df_path = '../../Dataset/test.csv/test.csv'
processed_test_path = '../output/processed_test.csv'

# feats_path = 'Ordinal_only_have_NANs.csv'
feats_path = 'Ordinal_only_NANs_not_replaced.csv'


top_n = 40

feats_df = pd.read_csv(feats_path)
feats = feats_df['col_name'].values.tolist()[:top_n]


In [12]:
processed_df = pd.read_csv(process_train_path)
X = processed_df.drop(['price_doc'], axis=1)
y = processed_df['price_doc']

X = X[feats]

In [13]:

def train(X, y):
    
    model = XGBRegressor(
                      n_estimators=xgb_params['n_estimators'],
                      device=xgb_params['device'],
                      objective=xgb_params['objective'],
                      eval_metric=xgb_params['eval_metric'],
                      enable_categorical=xgb_params['enable_categorical'],
                      
                      eta=xgb_params['eta'],
                      max_depth = xgb_params['max_depth'],
                    #   gamma = xgb_params['gamma'],
                    #   reg_alpha = xgb_params['reg_alpha'],
                    #   min_child_weight=xgb_params['min_child_weight'],
                      colsample_bytree=xgb_params['colsample_bytree']                     
                      )
  
    model.fit(X, y)

    return model

def test(model, X_test, y_test):
    pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, pred, squared=False)
    r2 = r2_score(y_test, pred)

    print(f"RMSE: {rmse}")
    print(f"R2: {r2}")

    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_test)

    shap.summary_plot(shap_values, X_test, plot_type="bar")

In [14]:
model = train(X, y)

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
[16:08:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\context.cc:44: No visible GPU is found, setting device to CPU.


In [15]:
test_df = pd.read_csv(test_df_path)
processed_test_df = pd.read_csv(processed_test_path)
processed_test_df.head()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,...,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,year,month
0,39.0,20.700000,2.0,9.0,0.0,1998.0,1,8.9,2.0,0.0,...,0,0,1,10,1,0,14,1,2015,6.0
1,79.2,32.660037,8.0,17.0,0.0,0.0,3,1.0,0.0,1.0,...,1,0,2,11,0,1,12,1,2015,6.0
2,40.5,25.100000,3.0,5.0,1.0,1960.0,2,4.8,1.0,0.0,...,4,0,10,21,0,10,71,11,2015,6.0
3,62.8,36.000000,17.0,17.0,0.0,2016.0,2,62.8,2.0,1.0,...,2,0,0,10,0,0,2,0,2015,6.0
4,40.0,40.000000,17.0,17.0,0.0,0.0,1,1.0,0.0,1.0,...,1,0,2,12,0,1,11,1,2015,6.0


In [16]:
pred = model.predict(processed_test_df[feats])
pred

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead


array([5800680.5, 8717462. , 5894417. , ..., 4565642. , 5721251.5,
       9609387. ], dtype=float32)

In [17]:
prediction_df = pd.DataFrame({
    'id': test_df['id'],
    'price_doc': pred
})

In [18]:
prediction_df.to_csv('xgb_pred.csv', index=False)